# Data Transformation

In this notebook, we look into some of the central data transformation techniques. For more data transformation techniques, see the book [Python for Data Analysis, 3E](https://wesmckinney.com/book/) by Wes McKinney.

In [1]:
import numpy as np
import pandas as pd

## Grouping and aggregation

Sometimes we want to calculate various functions on particular subgroups of a dataset. For instance, in the exercise last time, we asked the following question for the "adult" dataset: *"Is the average hours per week (worked) different across different marital-status groups?"*. That is, we asked for the average of `hours-per-week` for each categorical value of the variable `marital-status`. Such calculations are common and easily done using pandas.

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [ ]:
X

To get the mean hours-per-week per marital-status, we can take out the `hours-per-week` column and group it by the `marital-status` column:

In [ ]:
hgbm = X["hours-per-week"].groupby(X["marital-status"])
hgbm

This will give us a grouped object. We can now aggregate the `hours-per-week` for each marital status using various functions (methods) such as mean or median:

In [ ]:
hgbm.mean()

In [ ]:
hgbm.median()

We can also count how many cases there are in each group:

In [ ]:
hgbm.count()

Or get several describtive statics at once using `describe`:

In [ ]:
hgbm.describe()

We can also group by multiple categorical variables:

In [ ]:
hgbmg = X["hours-per-week"].groupby([X["marital-status"], X["sex"]])
hgbmg

In [ ]:
hgbmg.count()

In [ ]:
hgbmg.mean()

We can also group the entire dataframe by a categorical variable:

In [ ]:
Xg = X.groupby("marital-status")
Xg

We can then subset specific columns and aggregate those:

In [ ]:
Xg["age"].mean()

In [ ]:
Xg[["age", "hours-per-week"]].mean()

Or calculate aggregations of all nummeric columns:

In [ ]:
Xg.mean(numeric_only=True)

### Aggregating with abitrary functions

`mean`, `median`, ... etc. are some of the most common statistical functions and they are explicit methods on the DataFrames and Series. However, sometimes one wants to aggregate by another function that is not a method on the DataFrame or Series. This functionality is also possible in pandas.

For instance, you might want to calculate the range of a variable, that is the max value minus the min value. To do this, we first create the function we want to aggregate by (in this case range), and then use the `agg` method on the grouped object:

In [ ]:
def data_range(x):
    return (x.max() - x.min())

In [ ]:
Xg[["age"]].agg(data_range)

Or if we want to apply it on the entire dataframe we need the range function to check for numeric type

In [ ]:
from pandas.api.types import is_numeric_dtype

def data_range(x):
    if is_numeric_dtype(x):
        return (x.max() - x.min())
    else:
        return np.nan

Xg.agg(data_range)

You can also use `agg` to get the aggregation by multiple functions:

In [ ]:
Xg[["age"]].agg([np.mean, np.median, np.std])

Or if you want the columns named:

In [ ]:
Xg[["age"]].agg([("Mean age", np.mean), ("Median age", np.median), ("Standard deviatio of age", np.std)])

As these aggregation functions are also methods on the dataframe, as the warning tell us, we can just write `"mean"` instead of `np.mean` for instance:

In [ ]:
Xg[["age"]].agg([("Mean age", "mean"), ("Median age", "median"), ("Standard deviatio of age", "std")])

## Joins

Sometimes our data comes in multiple dataframes or tables, and we want to combine them into one for doing machine learning, for instance. We use joins just like in SQL to do this in python.

In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"], "data2": pd.Series(range(3), dtype="Int64")})

In [ ]:
df1

In [ ]:
df2

The `merge` function from Pandas do an inner join:

In [ ]:
pd.merge(df1, df2)

Specifying keys

In [ ]:
pd.merge(df1, df2, on="key")

If the key is named differently in the two dataframes

In [ ]:
df3 = df2.rename(columns={"key": "key_other"})
df3

In [ ]:
df2

In [ ]:
pd.merge(df1, df3, left_on="key", right_on="key_other")

Doing an outer, left or right join:

In [ ]:
pd.merge(df1, df2, on="key", how = "outer")

In [ ]:
pd.merge(df1, df2, on="key", how = "left")

In [ ]:
pd.merge(df1, df2, on="key", how = "right")

### Using indexes for joins

Sometimes the keys we want to join on are in the indexes of the data frames instead of in a column. To join, we can just turn the index into a column.

In [ ]:
df4 = df1.set_index('key')
df4

As you can see above, `df4` contains the keys in the index. We can move the indexes into a column using `reset_index`:

In [ ]:
df4 = df4.reset_index()
df4

Now we can merge as usual:

In [ ]:
pd.merge(df4, df2)

## Pivoting

Sometimes we want to make our dataframe wider or longer, that is move information between the columns names and the column values.

### Pivoting long to wide

In [ ]:
long_df = pd.DataFrame({"student_id" : [1,1,2,2,2,3,3],
                       "class" : ["algebra", "databases","algebra", "databases", "creative writing", "algebra", "databases"],
                       "grade" : [7, 10, 4, 2, 10, 4, 12]})

In [ ]:
long_df

In [ ]:
long_df.info()

We want to make it "wider", in the sense that we want one row per student, and then a column for each class.

In [ ]:
long_df.pivot(index = "student_id", columns="class", values="grade")

Note that the ´student_id´ column is turned into the index. If we want to keep it as a column we can do:

In [ ]:
long_df.pivot(index = "student_id", columns="class", values="grade").reset_index()

The metod `pivot_table` is a more general method that allow for more functionality if needed.

### Pivoting wide to long

In [ ]:
wide_df = pd.DataFrame({"Country" : ["Denmark", "Denmark", "Sweden", "Sweden"],
                       "Type" : ["population", "infected", "population", "infected",],
                       "2001" : [5000, 1, 9500, 2],
                       "2002" : [5050, 3, 9550, 4],
                       "2003" : [5100, 6, 9650, 8],
                       "2004" : [5150, 10, 9700, 9]})

In [ ]:
wide_df

In [ ]:
wide_df.melt(id_vars = ["Country", "Type"], value_vars = ["2001", "2002", "2003", "2004"])

You might want to rename the `variable` and  `value` column afterwards:

In [ ]:
wide_df.melt(id_vars = ["Country", "Type"], value_vars = ["2001", "2002", "2003", "2004"]).rename(columns={"variable": "Year", "value": "Size"})

## Creating new columns from mapping

We can easily create a new column with a high-level categorization from a column that might have more values using the mapping functionality:

In [ ]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
# A dictionary mapping
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

In [ ]:
data["food"].map(meat_to_animal)

In [ ]:
data["animal"] = data["food"].map(meat_to_animal)
data

In [ ]:
# Defining a function to do the mapping (here based on the dictionary!)
def get_animal(x):
    return meat_to_animal[x]

In [ ]:
data = data.drop("animal", axis = 1)
data

In [ ]:
data["animal"] = data["food"].map(get_animal)
data

## Transforming a categorical variable into dummy variables

In [ ]:
data

Let us turn the categorical variable `animal` into dummy variables:

In [ ]:
pd.get_dummies(data["animal"])

If we want the values to be 0 and 1 (int or float):

In [ ]:
pd.get_dummies(data["animal"], dtype = "int")

If we want to add the dummies to the original dataframe we can do the following steps:

In [ ]:
dummies = pd.get_dummies(data["animal"], prefix="dummy", dtype=float)
dummies

In [ ]:
data_with_dummies = data.join(dummies)
data_with_dummies

We might want to drop the original column:

In [ ]:
data_with_dummies = data_with_dummies.drop("animal", axis = "columns")
data_with_dummies

**Note: When training machine learning models, it is often problematic if multiple column perfectly correlates. Here `dummy_cow` perfectly correlates with the two columns `dummy_pig` and `dummy_salmon`, in the sense that `dummy_cow = 1 - dummy_pig - dummy_salmon`. Thus one usually drops one of the dummy columns.**